<a href="https://colab.research.google.com/github/dbaglodi/AI-For-AccessCORPS/blob/main/document_remediation_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook processes `.docx` and `.pptx` files to extract images and their associated alt text for remediation.

# Instructions:
0. Create a Google API Key and add your key to Colab Secrets under the name "GOOGLE_API_KEY" (Follow "Create an API Key" and "Add your key to Colab Secrets" from this link: https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb#scrollTo=mhFKmRmxi5B-)
1. Run the Setup section.
2. Expand the section(s) for which you wish to upload a file (Word Documents or PowerPoint Documents).
3. Go to the desired subsection and run the corresponding cell, where you will upload your `.docx` or `.pptx` files into the environment.
4. Run each code cell in order, following any written instructions.
5. If applicable, review the generated image previews and alt text suggestions.
6. Edit or confirm alt text as needed.
7. Download the remediated file with updated alt text embedded.

**Note:** Ensure your files are in the correct format (`.docx` or `.pptx`) and contain images for the best results.

# Setup (Required)
Run this section before going to any other section.

In [ ]:
# @title
!apt-get update
!apt-get install -y libreoffice poppler-utils ghostscript
!pip install lxml python-docx python-pptx google-generativeai Pillow numpy opencv-python-headless pdf2img

In [ ]:
# @title
import os
import re
import zipfile
from lxml import etree
from docx import Document
from collections import Counter
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE
from PIL import Image, WmfImagePlugin
import io
import google.generativeai as genai
from google.colab import files
import time

In [ ]:
# @title
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
gemini_model = 'gemini-2.0-flash'

In [ ]:
def wmf_handler(fp, filename):
    """Handler to open WMF images."""
    try:
        return WmfImagePlugin.WmfImageFile(fp)
    except Exception as e:
        print(f"Error processing WMF file {filename}: {e}")
        return None

WmfImagePlugin.register_handler(wmf_handler)

In [ ]:
def retry_operation(operation, retries=3, delay=30, default=None):
    """Helper function to retry operations with exponential backoff."""
    for i in range(retries):
        try:
            return operation()
        except Exception as e:
            if i == retries - 1:
                print(f"Operation failed after {retries} attempts: {str(e)}")
                return default
            print(f"Operation failed: {str(e)}. Retrying in {delay} seconds...")
            time.sleep(delay)
            delay *= 2  # Exponential backoff

## Define Functions

In [ ]:
def tag_image(image_bytes, context_text="", slide_screenshot=None):
    """Classify the image using Google Generative AI."""
    # Convert bytes to PIL Image for processing
    image = Image.open(io.BytesIO(image_bytes))
    if image.mode != 'RGB':
        image = image.convert('RGB')

    additional_context = ""
    if slide_screenshot is not None:
        # Convert slide screenshot bytes to PIL Image for analysis
        slide_screenshot_img = Image.open(io.BytesIO(slide_screenshot))
        additional_context += "Slide Context: The image is part of a slide with the following visual context. "

    classification_prompt = f"""Using the image provided, classify it into the following categories based on its primary content and purpose.

{additional_context}
Nearby Text Context: "{context_text}"

Categories:
Graph: Visual representations of data, typically with axes and labeled points, such as line graphs, bar graphs, and scatter plots. They display numerical data trends over time or across categories.
Chart: Broad category that includes pie charts, flow charts, and similar visuals. Charts are used to show relationships, hierarchies, or proportions among different components or processes.
Map: Geographical representations showing locations, terrain, or routes. Maps can include details like topography, political boundaries, or population density.
Diagram: Illustrative visualizations explaining concepts, structures, or processes, such as circuit diagrams, organizational charts, or flow diagrams. Diagrams often break down complex ideas.
Table: Gridded arrangements of data in rows and columns, often with headings. Tables are used for easy lookup and comparison of related information.
Photograph: Real-life images that capture a scene, object, or event, often used for documentation or visual reference.
Text: Visual representations of written information, often without additional visual elements. Text images are used to convey information directly in written form rather than through data or graphical representations.
Screenshot: Captured images from a digital interface, such as a website, software, or application screen, usually to illustrate a particular function or feature.
Equation: Visuals showing mathematical or scientific equations, formulas, or expressions.
Other: An image that does not serve any of the above purposes.

Provide your output as a comma-separated list of labels, such as "Graph, Diagram". Do not provide any explanation, and do not add brackets to the list."""

    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    model = genai.GenerativeModel(gemini_model)

    # Include both image and slide screenshot if available
    content_inputs = [classification_prompt, image]
    if slide_screenshot is not None:
        content_inputs.append(slide_screenshot_img)

    response = model.generate_content(content_inputs)
    response = response.text.strip()
    response = [r.strip() for r in response.split(',')]
    return response

In [ ]:
def generate_alt_text(image_bytes, context_text, existing_alt, tags, presentation_metadata=None, slide_num=None, slide_screenshot=None):
    """Generate alt text using Google Generative AI with enhanced context."""
    image = Image.open(io.BytesIO(image_bytes))
    if image.mode != 'RGB':
        image = image.convert('RGB')

    # Add presentation-level context if available
    presentation_context = ""
    if presentation_metadata:
        presentation_context = f"""
        Presentation Title: {presentation_metadata.get('title', '')}
        Presentation Summary: {presentation_metadata.get('summary', '')}
        """

        # Add slide-specific context if available
        if slide_num is not None and 'slide_headers' in presentation_metadata:
            if 0 <= slide_num-1 < len(presentation_metadata['slide_headers']):
                slide_header = presentation_metadata['slide_headers'][slide_num-1]
                if slide_header:
                    presentation_context += f"\nSlide Title: {slide_header}"

    # Prepare additional visual context description
    visual_context = ""
    if slide_screenshot is not None:
        # Convert slide screenshot to PIL Image for analysis
        slide_screenshot_img = Image.open(io.BytesIO(slide_screenshot))
        visual_context = "The image is part of a slide with additional visual context that provides further information about the content."

    prompt = f"""Generate a concise, descriptive alt text for this image that has been classified with the following tags: {tags}.

    {presentation_context}

    {visual_context}

    Slide Context: This image appears with the following text: {context_text}
    Existing alt text: {existing_alt}

    Important guidelines:
    1. Focus on describing key visual elements and their relationship to the context
    2. Use clear, academic language appropriate for the content
    3. Keep the description under 125 characters
    4. If the image shows a diagram or figure, describe its key components and purpose
    5. Include relevant technical terms from the context when appropriate
    6. Consider how this image fits into the overall presentation theme

    Please provide just the alt text without any additional commentary."""

    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    model = genai.GenerativeModel(gemini_model)

    # Include both image and slide screenshot if available
    content_inputs = [prompt, image]
    if slide_screenshot is not None:
        content_inputs.append(slide_screenshot_img)

    response = model.generate_content(content_inputs)
    return response.text.strip()

In [ ]:
def generate_long_description(image_bytes, context_text, existing_alt, tags, presentation_metadata=None, slide_num=None, slide_screenshot=None):
    """Generate alt text using Google Generative AI with enhanced context."""
    image = Image.open(io.BytesIO(image_bytes))
    if image.mode != 'RGB':
        image = image.convert('RGB')

    # Add presentation-level context if available
    presentation_context = ""
    if presentation_metadata:
        presentation_context = f"""
        Presentation Title: {presentation_metadata.get('title', '')}
        Presentation Summary: {presentation_metadata.get('summary', '')}
        """

        # Add slide-specific context if available
        if slide_num is not None and 'slide_headers' in presentation_metadata:
            if 0 <= slide_num-1 < len(presentation_metadata['slide_headers']):
                slide_header = presentation_metadata['slide_headers'][slide_num-1]
                if slide_header:
                    presentation_context += f"\nSlide Title: {slide_header}"

    # Prepare additional visual context description
    visual_context = ""
    if slide_screenshot is not None:
        # Convert slide screenshot to PIL Image for analysis
        slide_screenshot_img = Image.open(io.BytesIO(slide_screenshot))
        visual_context = "The image is part of a slide with additional visual context that provides further information about the content."

    prompt = f"""Generate a description for this image that has been classified with the following tags: {tags}.

    {presentation_context}

    {visual_context}

    Slide Context: This image appears with the following text: {context_text}
    Alt text description: {existing_alt}

    Important guidelines:
    1. Focus on describing key visual elements and their relationship to the context
    2. Use clear, academic language appropriate for the content
    3. Have the description be at least 150 characters
    4. If the image shows a diagram or figure, describe its key components and purpose
    5. Include relevant technical terms from the context when appropriate
    6. Consider how this image fits into the overall presentation theme

    Please provide just the alt text without any additional commentary."""

    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    model = genai.GenerativeModel(gemini_model)

    # Include both image and slide screenshot if available
    content_inputs = [prompt, image]
    if slide_screenshot is not None:
        content_inputs.append(slide_screenshot_img)

    response = model.generate_content(content_inputs)
    return response.text.strip()

In [ ]:
# @title
def preprocess_document(docx_file, include_alt_texts=False):
    """
    Process a Word document to generate a summary and extract key themes.

    Args:
        docx_file (str): Path to the Word document
        include_alt_texts (bool): Whether to include alt texts in the analysis

    Returns:
        dict: Contains document summary and other metadata
    """
    try:
        print("Preprocessing document to generate summary...")
        document = Document(docx_file)

        # Extract text from paragraphs
        all_text = []
        for para in document.paragraphs:
            text = para.text.strip()
            if text:
                all_text.append(text)

        # Extract headings
        headings = [para.text.strip() for para in document.paragraphs if para.style.name.startswith("Heading")]

        # Extract alt texts from images if requested
        all_alt_texts = []
        if include_alt_texts:
            for rel in document.part.rels:
                if "image" in document.part.rels[rel].target_ref:
                    img = document.part.rels[rel].target_part
                    if hasattr(img, "alt_text") and img.alt_text:
                        all_alt_texts.append(img.alt_text.strip())

        # Extract keywords using simple frequency analysis
        words = ' '.join(all_text).lower()
        stop_words = set(['a', 'an', 'the', 'and', 'or', 'but', 'is', 'are', 'was', 'were',
                         'in', 'on', 'at', 'to', 'for', 'with', 'by', 'of', 'this', 'that'])
        word_list = re.findall(r'\b[a-z]{3,}\b', words)
        word_list = [word for word in word_list if word not in stop_words]
        word_freq = Counter(word_list)
        keywords = [word for word, count in word_freq.most_common(10)]

        # Generate summary using Gemini
        if 'GOOGLE_API_KEY' in os.environ:
            try:
                # Create a concise description of the document
                outline = "\n".join([f"Headings: {headings}"])

                # Include alt texts in the summary prompt if available
                alt_text_info = ""
                if include_alt_texts and all_alt_texts:
                    alt_text_summary = "\n".join(all_alt_texts[:20])  # Limit to avoid too much text
                    alt_text_info = f"""
                    Alt texts from document images:
                    {alt_text_summary}
                    """

                summary_prompt = f"""Generate a concise summary (150 words max) of this Word document.

                Frequent keywords: {', '.join(keywords)}

                Number of paragraphs: {len(document.paragraphs)}

                Outline: {outline}

                {alt_text_info}

                Based on this information, provide:
                1. A brief overview of what this document appears to be about
                2. The likely target audience
                3. The main themes or key points

                Format your response as a single paragraph that would help someone understand the context of this document without being too verbose."""

                genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
                model = genai.GenerativeModel(gemini_model)
                response = model.generate_content(summary_prompt)
                summary = response.text.strip()
            except Exception as e:
                print(f"Error generating summary: {str(e)}")
                summary = f"A document with {len(document.paragraphs)} paragraphs, focusing on {', '.join(keywords[:5])}."
        else:
            summary = f"A document with {len(document.paragraphs)} paragraphs, focusing on {', '.join(keywords[:5])}."

        document_metadata = {
            "title": os.path.basename(docx_file),
            "paragraph_count": len(document.paragraphs),
            "summary": summary,
            "keywords": keywords,
            "headings": headings,
            "alt_texts": all_alt_texts if include_alt_texts else []
        }

        print(f"Generated document summary: {summary}")
        return document_metadata

    except Exception as e:
        print(f"Error preprocessing document: {str(e)}")
        return {
            "title": os.path.basename(docx_file),
            "summary": "Unable to generate document summary.",
            "keywords": []
        }

def process_docx(docx_file, reprocess=False, user_input=True):
    """Process DOCX file and update alt text in-place."""
    temp_dir = 'temp_docx'
    os.makedirs(temp_dir, exist_ok=True)

    # Extract the docx
    with zipfile.ZipFile(docx_file, 'r') as docx_zip:
        docx_zip.extractall(temp_dir)

    # Generate metadata of the document
    if reprocess:
        print("Reprocessing document metadata...")
        document_metadata = preprocess_document(docx_file, include_alt_texts=True)
    else:
        print("Processing document metadata...")
        document_metadata = preprocess_document(docx_file)

    # Parse document and relationships
    document_xml_path = os.path.join(temp_dir, 'word', 'document.xml')
    rels_xml_path = os.path.join(temp_dir, 'word', '_rels', 'document.xml.rels')

    tree = etree.parse(document_xml_path)
    root = tree.getroot()
    rels_tree = etree.parse(rels_xml_path)
    rels_root = rels_tree.getroot()

    namespaces = {
        'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main',
        'a': 'http://schemas.openxmlformats.org/drawingml/2006/main',
        'r': 'http://schemas.openxmlformats.org/officeDocument/2006/relationships',
        'wp': 'http://schemas.openxmlformats.org/drawingml/2006/wordprocessingDrawing',
        'pic': 'http://schemas.openxmlformats.org/drawingml/2006/picture',
    }

    classifications = {}

    # Process each image
    for paragraph in root.xpath('//w:p', namespaces=namespaces):
        paragraph_text = ''.join(paragraph.xpath('.//w:t/text()', namespaces=namespaces)).strip()

        for drawing in paragraph.xpath('.//w:drawing', namespaces=namespaces):
            blip_elements = drawing.xpath('.//a:blip', namespaces=namespaces)
            if not blip_elements:
                continue

            blip = blip_elements[0]
            rel_id = blip.get('{http://schemas.openxmlformats.org/officeDocument/2006/relationships}embed')

            # Find image path
            target = None
            for rel in rels_root:
                if rel.get('Id') == rel_id:
                    target = rel.get('Target')
                    break

            if not target:
                continue

            image_path = os.path.join(temp_dir, 'word', target)
            if not os.path.exists(image_path):
                continue

            # Get existing alt text
            existing_alt = ''
            docPr_elements = drawing.xpath('.//wp:docPr', namespaces=namespaces)
            if docPr_elements:
                existing_alt = docPr_elements[0].get('descr', '')
                print(f"Existing alt text: {existing_alt}")

            # Process image
            with open(image_path, 'rb') as img_file:
                image_bytes = img_file.read()

            #Show image
            image = Image.open(io.BytesIO(image_bytes))
            display(image)

            # Classify image and generate new alt text
            num_tries = 3
            sleep_time = 10
            for i in range(num_tries):
                try:
                  #classification = classify_image(image_bytes)
                  tags = tag_image(image_bytes, paragraph_text)
                  #print(f"Classified image as {classification}")
                  print(f"Tagged image as {tags}")
                  for tag in tags:
                    if tag not in classifications:
                      classifications[tag] = 0
                    classifications[tag] += 1
                except Exception as e:
                  if i == num_tries - 1:
                    print(f"Error classifying image: {str(e)}")
                    #classification = "Other"
                    tags = ["Other"]
                  else:
                    print(f"Error classifying image: {str(e)}. Trying again in {sleep_time} seconds...")
                    time.sleep(sleep_time)
                  continue
                else:
                  break
            for i in range(num_tries):
                try:
                  new_alt_text = generate_alt_text(
                      image_bytes,
                      paragraph_text,
                      existing_alt,
                      #classification
                      tags,
                      document_metadata
                  )
                  print(f"Generated new alt text: {new_alt_text}")
                except Exception as e:
                  if i == num_tries - 1:
                    print(f"Error generating alt text: {str(e)}")
                    new_alt_text = existing_alt
                  else:
                    print(f"Error generating alt text: {str(e)}. Trying again in {sleep_time} seconds...")
                    time.sleep(sleep_time)
                  continue
                else:
                  break
            # Handle user input
            if user_input:
                alt = input("Press enter to input the new alt text. Enter 1 to keep the old alt text. Enter 2 to mark as decorative. Otherwise, enter your own alt text: \n")
                if alt.strip() == "1":
                    new_alt_text = existing_alt
                elif alt.strip() == "2":
                    new_alt_text = ""
                elif alt.strip() != "":
                    new_alt_text = alt
            # Update alt text in the document
            if docPr_elements:
                docPr_elements[0].set('descr', new_alt_text)

            # Generate long description
            long_description = retry_operation(
                lambda: generate_long_description(
                    image_bytes,
                    paragraph_text,
                    new_alt_text,
                    tags,
                    document_metadata
                ),
                retries=3,
                delay=30,
                default=new_alt_text
            )
            print(f"Generated long description: {long_description}")

    # Save modified document
    tree.write(document_xml_path, encoding='UTF-8', xml_declaration=True)

    print(f"Classifications: {classifications}")

    # Create new docx with updated content
    output_docx = f'updated_{docx_file}'
    with zipfile.ZipFile(output_docx, 'w') as outzip:
        for root_dir, _, files in os.walk(temp_dir):
            for file in files:
                file_path = os.path.join(root_dir, file)
                arcname = os.path.relpath(file_path, temp_dir)
                outzip.write(file_path, arcname)

    # Clean up
    import shutil
    shutil.rmtree(temp_dir)

    return output_docx

In [ ]:
# @title
import json
from pathlib import Path
import io
from PIL import Image, WmfImagePlugin
import numpy as np
import cv2
import google.generativeai as genai
import os
import time
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE
from pptx.util import Pt, Inches
from pptx.dml.color import RGBColor
from pptx.enum.dml import MSO_THEME_COLOR
from collections import Counter, defaultdict
import re
from pdf2image import convert_from_path
import subprocess
import tempfile


def extract_slide_text(slide):
    """Extract all text from a slide"""
    text_content = []
    for shape in slide.shapes:
        if hasattr(shape, "text"):
            text = shape.text.strip()
            if text:
                text_content.append(text)
    return " ".join(text_content)

def extract_slide_alt_texts(slide):
    """Extract all alt texts from a slide's images"""
    alt_texts = []
    for shape in slide.shapes:
        if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
            alt_text = shape._element._nvXxPr.cNvPr.attrib.get("descr", "").strip()
            if alt_text:
                alt_texts.append(alt_text)
    return alt_texts

def preprocess_presentation(pptx_file, include_alt_texts=False):
    """
    Process the entire presentation to generate a summary and extract key themes.

    Args:
        pptx_file (str): Path to the PowerPoint file
        include_alt_texts (bool): Whether to include alt texts in the analysis

    Returns:
        dict: Contains presentation summary and other metadata
    """
    try:
        print("Preprocessing presentation to generate summary...")
        presentation = Presentation(pptx_file)

        # Extract title from the first slide if available
        presentation_title = ""
        if len(presentation.slides) > 0:
            first_slide = presentation.slides[0]
            for shape in first_slide.shapes:
                if hasattr(shape, "text") and shape.text.strip():
                    presentation_title = shape.text.strip()
                    break

        # Collect all text from all slides
        all_text = []
        slide_headers = []
        all_alt_texts = []
        hidden_slides = []

        for slide_num, slide in enumerate(presentation.slides, start=1):
            slide_text = extract_slide_text(slide)
            all_text.append(slide_text)

            # Check if slide is hidden
            if slide._element.get("show") == "0":
                hidden_slides.append(slide_num)
                slide._element.set("show", "1")
                presentation.save(pptx_file)

            # Try to extract slide title/header
            header = ""
            for shape in slide.shapes:
                if hasattr(shape, "text") and shape.text.strip():
                    header = shape.text.strip().split('\n')[0]  # Take first line as header
                    break
            slide_headers.append(header)

            # Extract alt texts if requested
            if include_alt_texts:
                slide_alt_texts = extract_slide_alt_texts(slide)
                all_alt_texts.extend(slide_alt_texts)

        # Extract keywords using simple frequency analysis
        words = ' '.join(all_text).lower()
        # Remove common stop words
        stop_words = set(['a', 'an', 'the', 'and', 'or', 'but', 'is', 'are', 'was', 'were',
                         'in', 'on', 'at', 'to', 'for', 'with', 'by', 'of', 'this', 'that'])
        word_list = re.findall(r'\b[a-z]{3,}\b', words)
        word_list = [word for word in word_list if word not in stop_words]
        word_freq = Counter(word_list)
        keywords = [word for word, count in word_freq.most_common(10)]

        # Count image types
        image_types = defaultdict(int)
        for slide in presentation.slides:
            for shape in slide.shapes:
                if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
                    image_types["total"] += 1

        # Generate summary using Gemini
        if 'GOOGLE_API_KEY' in os.environ:
            try:
                # Create a concise description of the presentation
                slide_outline = "\n".join([f"Slide {i+1}: {header}" for i, header in enumerate(slide_headers) if header])

                # Include alt texts in the summary prompt if available
                alt_text_info = ""
                if include_alt_texts and all_alt_texts:
                    alt_text_summary = "\n".join(all_alt_texts[:20])  # Limit to avoid too much text
                    alt_text_info = f"""
                    Alt texts from presentation images:
                    {alt_text_summary}
                    """

                summary_prompt = f"""Generate a concise summary (150 words max) of this PowerPoint presentation.

                Title: {presentation_title}

                Number of slides: {len(presentation.slides)}

                Slide outline:
                {slide_outline}

                Frequent keywords: {', '.join(keywords)}

                {alt_text_info}

                Based on this information, provide:
                1. A brief overview of what this presentation appears to be about
                2. The likely target audience
                3. The main themes or key points

                Format your response as a single paragraph that would help someone understand the context of this presentation without being too verbose."""

                genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
                model = genai.GenerativeModel(gemini_model)
                response = model.generate_content(summary_prompt)
                summary = response.text.strip()
            except Exception as e:
                print(f"Error generating summary: {str(e)}")
                summary = f"A {len(presentation.slides)}-slide presentation titled '{presentation_title}' covering topics related to {', '.join(keywords[:5])}."
        else:
            summary = f"A {len(presentation.slides)}-slide presentation titled '{presentation_title}' covering topics related to {', '.join(keywords[:5])}."

        presentation_metadata = {
            "title": presentation_title,
            "slide_count": len(presentation.slides),
            "summary": summary,
            "keywords": keywords,
            "slide_headers": slide_headers,
            "image_count": image_types["total"],
            "alt_texts": all_alt_texts if include_alt_texts else [],
            "hidden_slides": hidden_slides
        }

        print(f"Generated presentation summary: {summary}")
        return presentation_metadata

    except Exception as e:
        print(f"Error preprocessing presentation: {str(e)}")
        return {
            "title": os.path.basename(pptx_file),
            "summary": "Unable to generate presentation summary.",
            "keywords": []
        }


def generate_slide_title(slide_text, presentation_metadata=None, slide_screenshot=None):
    """Generate a slide title using the slide text and presentation context."""
    # Add presentation-level context if available
    presentation_context = ""
    if presentation_metadata:
        presentation_context = f"""
        Presentation Title: {presentation_metadata.get('title', 'Unknown')}
        Presentation Summary: {presentation_metadata.get('summary', 'Not available')}
        Keywords: {', '.join(presentation_metadata.get('keywords', []))}
        """

    # Prepare additional visual context description
    visual_context = ""
    if slide_screenshot is not None:
        # Convert slide screenshot to PIL Image for analysis
        slide_screenshot_img = Image.open(io.BytesIO(slide_screenshot))
        visual_context = "The slide contains visual elements that provide additional context to the textual content."

    prompt = f"""Generate a concise, descriptive title for a PowerPoint slide based on the following context:

    {presentation_context}

    {visual_context}

    Slide Text Content: {slide_text}

    Important guidelines:
    1. The title should be 3-7 words
    2. Capture the main topic or purpose of the slide
    3. Use clear, professional language appropriate for a presentation
    4. Make the title stand out as a meaningful section header
    5. Consider how this title fits into the overall presentation theme
    6. Do not use punctuation at the end of the title

    Please provide just the title without any additional commentary."""

    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    model = genai.GenerativeModel(gemini_model)

    # Include both slide text and slide screenshot if available
    content_inputs = [prompt]
    if slide_screenshot is not None:
        content_inputs.append(slide_screenshot_img)

    response = model.generate_content(prompt)
    return response.text.strip()

def add_hidden_title_to_slide(slide, title_text):
    """
    Adds a title to a PowerPoint slide and moves it off-screen to hide it.

    :param slide: The slide to which the title should be added.
    :param title_text: The text to set as the title.
    """
    # Add a title placeholder if not already present
    title = slide.shapes.title
    if title is None:
        title = slide.shapes.add_textbox(Inches(0), Inches(0), Inches(1), Inches(1))
        text_frame = title.text_frame
    else:
        text_frame = title.text_frame

    # Set the title text
    text_frame.clear()  # Clear any existing text
    title.text = title_text
    slide.name = title_text

    # Move the title off-screen
    title.top = -Inches(1)   # Move off-screen to the top

    return slide

def has_title(slide):
    """Check if a slide has a visible title."""
    if slide.shapes.title and slide.shapes.title.text.strip():
        return True

    # Also check for any shape that might be functioning as a title
    for shape in slide.shapes:
        if hasattr(shape, "text"):
            # If there's a shape with text near the top of the slide, it might be a title
            text = shape.text.strip()
            if text and shape.top < Pt(100) and len(text.split()) <= 10:
                return True

    return False

pptx_to_images_result = None

def convert_pptx_to_images(pptx_file):
    """ Converts PPTX to PDF using LibreOffice and extracts slide images. """
    global pptx_to_images_result
    if pptx_to_images_result is not None:
        return pptx_to_images_result
    with tempfile.TemporaryDirectory() as temp_dir:
        pdf_path = os.path.join(temp_dir, "output.pdf")

        # Convert PPTX to PDF using LibreOffice
        subprocess.run(
            ["libreoffice", "--headless", "--convert-to", "pdf", "--outdir", temp_dir, pptx_file],
            check=True
        )

        # Find the converted PDF file
        pdf_files = [f for f in os.listdir(temp_dir) if f.endswith(".pdf")]
        if not pdf_files:
            raise RuntimeError("Failed to convert PowerPoint to PDF.")

        pdf_path = os.path.join(temp_dir, pdf_files[0])

        # Convert the PDF pages to images
        pptx_to_images_result = convert_from_path(pdf_path)
        return pptx_to_images_result

def take_slide_screenshot(pptx_file, slide_number):
    """
    Takes a screenshot of a specified slide in a PowerPoint file and returns it as image bytes.
    Uses LibreOffice (headless) to convert PPTX to PDF and extracts the specified slide as an image.
    """

    # Validate slide number
    presentation = Presentation(pptx_file)
    #if hidden:
    #    presentation.slides[slide_number-1]._element.set("show", "1")
    #    presentation.save(pptx_file)
    total_slides = len(presentation.slides)

    if slide_number < 1 or slide_number > total_slides:
        raise ValueError(f"Slide {slide_number} is out of range (1-{total_slides}).")

    # Convert PPTX to images and get the correct slide
    slide_images = convert_pptx_to_images(pptx_file)

    if slide_number > len(slide_images):
        raise ValueError(f"Slide {slide_number} not found in converted PDF.")

    slide_image = slide_images[slide_number - 1]  # Convert to zero-based index

    # Save image to bytes
    img_bytes = io.BytesIO()
    slide_image.save(img_bytes, format="PNG")
    img_bytes.seek(0)

    return img_bytes.read()

def process_pptx(pptx_file, start_slide=None, end_slide=None, resume_from_checkpoint=True, reprocess=False, user_input=True, classifications=None):
    """
    Process PowerPoint file and update alt text for a specific range of slides.
    Now also generates and adds hidden titles for slides missing titles.
    """
    try:
        # Verify Google API key is set
        if 'GOOGLE_API_KEY' not in os.environ:
            raise ValueError("GOOGLE_API_KEY environment variable is not set")

        # Generate presentation summary
        if reprocess:
            presentation_metadata = preprocess_presentation(pptx_file, include_alt_texts=True)
        else:
          presentation_metadata = preprocess_presentation(pptx_file)

        # Load the presentation
        presentation = Presentation(pptx_file)
        total_slides = len(presentation.slides)

        # Validate and adjust slide range
        if start_slide is None:
            start_slide = 1
        if end_slide is None:
            end_slide = total_slides

        # Validate slide range
        if not 1 <= start_slide <= total_slides:
            raise ValueError(f"Invalid slide range. Presentation has {total_slides} slides.")
        if not 1 <= end_slide <= total_slides:
            end_slide = total_slides
        if start_slide > end_slide:
            raise ValueError("Start slide cannot be greater than end slide.")

        print(f"Will process slides {start_slide} to {end_slide} out of {total_slides} total slides")

        # Load existing progress if any
        checkpoint_file = f'checkpoint_{Path(pptx_file).stem}_{start_slide}_{end_slide}.json'
        current_slide = start_slide

        if Path(checkpoint_file).exists() and resume_from_checkpoint:
            with open(checkpoint_file, 'r') as f:
                checkpoint_data = json.load(f)
                classifications = checkpoint_data['classifications']
                current_slide = checkpoint_data['last_slide'] + 1
                #hidden_slides = checkpoint_data['hidden_slides']
                if current_slide > end_slide:
                    print("Previous processing completed. Starting fresh.")
                    current_slide = start_slide
                    #classifications = {}
                else:
                    print(f"Resuming from slide {current_slide}")
        else:
            #classifications = {}
            #hidden_slides = []
            pass

        # Process each slide
        total_images = 0
        titles_added = 0

        for slide_num, slide in enumerate(presentation.slides, start=1):
            # Skip slides outside our range
            if slide_num < start_slide or slide_num > end_slide:
                continue

            # Skip slides we've already processed when resuming
            if slide_num < current_slide:
                continue

            try:
                print(f"Processing slide {slide_num}/{total_slides} (in range {start_slide}-{end_slide})")

                # Extract slide text for context
                slide_text = extract_slide_text(slide)
                print(f"Extracted text from slide {slide_num}")

                displayed_slide = False


                # Check if slide is hidden
                hidden = False
                if slide._element.get("show") == "0":
                  hidden = True
                  print(f"Slide {slide_num} is hidden")
                  #hidden_slides.append(slide_num)

                # Check if slide is missing a title and generate one if needed
                modified = False
                if not has_title(slide):
                    print(f"Slide {slide_num} is missing a title. Generating one...")

                    #Show full slide
                    #if hidden:
                    #    slide._element.set("show", "1")
                    slide_screenshot = take_slide_screenshot(pptx_file, slide_num)
                    print(f"Showing slide {slide_num}")
                    display(Image.open(io.BytesIO(slide_screenshot)))
                    displayed_slide = True

                    # Generate a title using the slide text context
                    generated_title = retry_operation(
                        lambda: generate_slide_title(slide_text, presentation_metadata, slide_screenshot),
                        retries=3,
                        delay=30,
                        default=f"Slide {slide_num}"
                    )
                    print(f"Generated title: {generated_title}")
                    if user_input:
                        title_input = input("Press enter to use the generated title. Otherwise, enter your own title: \n")

                        if title_input.strip() != "":
                            generated_title = title_input

                    # Add the title as a hidden element
                    add_hidden_title_to_slide(slide, generated_title)
                    print(f"Added hidden title to slide {slide_num}: '{generated_title}'")

                    # Update the presentation metadata with the new title
                    if 'slide_headers' in presentation_metadata and slide_num-1 < len(presentation_metadata['slide_headers']):
                        presentation_metadata['slide_headers'][slide_num-1] = generated_title

                    titles_added += 1
                    modified = True

                # Process each shape in the slide (existing image processing code)
                for shape in slide.shapes:
                    #print(shape.shape_type)
                    if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
                        try:
                            # Process image (existing code)
                            image = shape.image
                            image_bytes = image.blob

                            #Show full slide
                            if not displayed_slide:
                                #if not hidden:
                                #  slide._element.set("show", "1")
                                slide_screenshot = take_slide_screenshot(pptx_file, slide_num)
                                print(f"Showing slide {slide_num}")
                                display(Image.open(io.BytesIO(slide_screenshot)))
                                displayed_slide = True

                            # Show image
                            img = Image.open(io.BytesIO(image_bytes))
                            display(img)

                            # Get existing alt text
                            existing_alt = shape._element._nvXxPr.cNvPr.attrib.get("descr", "")
                            print(f"Processing image on slide {slide_num} (current alt text: {existing_alt})")

                            # Tag image with retries
                            tags = retry_operation(
                                lambda: tag_image(image_bytes, slide_text, slide_screenshot),
                                retries=3,
                                delay=30,
                                default=["Other"]
                            )
                            print(f"Tagged image as: {tags}")

                            if classifications is not None:
                                for tag in tags:
                                    if tag not in classifications:
                                        classifications[tag] = 1
                                    else:
                                        classifications[tag] += 1

                            # Generate new alt text with retries and enhanced context
                            new_alt_text = retry_operation(
                                lambda: generate_alt_text(
                                    image_bytes,
                                    slide_text,
                                    existing_alt,
                                    tags,
                                    presentation_metadata,
                                    slide_num,
                                    slide_screenshot
                                ),
                                retries=3,
                                delay=30,
                                default=existing_alt
                            )
                            print(f"Generated new alt text: {new_alt_text}")
                            if user_input:
                                alt = input("Press enter to input the new alt text. Enter 1 to keep the old alt text. Enter 2 to mark as decorative. Otherwise, enter your own alt text: \n")
                                change = True
                                if alt.strip() == "1":
                                    change = False
                                elif alt.strip() == "2":
                                    new_alt_text = ""
                                elif alt.strip() != "":
                                    new_alt_text = alt
                                # Update alt text
                                if change:
                                    shape._element._nvXxPr.cNvPr.set('descr', new_alt_text)
                                    modified = True
                                    total_images += 1
                            else:
                              shape._element._nvXxPr.cNvPr.set('descr', new_alt_text)
                              modified = True
                              total_images += 1

                            # Generate long description
                            long_description = retry_operation(
                                lambda: generate_long_description(
                                    image_bytes,
                                    slide_text,
                                    new_alt_text,
                                    tags,
                                    presentation_metadata,
                                    slide_num,
                                    slide_screenshot
                                ),
                                retries=3,
                                delay=30,
                                default=new_alt_text
                            )
                            print(f"Generated long description: {long_description}")

                        except Exception as e:
                            print(f"Error processing image on slide {slide_num}: {str(e)}")
                            continue

                # Save checkpoint after each slide if we modified anything
                if modified:
                    temp_output = f'temp_{Path(pptx_file).stem}_slide_{slide_num}.pptx'
                    presentation.save(temp_output)
                    print(f"Saved temporary file: {temp_output}")

                    # Save checkpoint data
                    checkpoint_data = {
                        'last_slide': slide_num,
                        'classifications': classifications,
                        'total_images': total_images,
                        'titles_added': titles_added,
                        #'hidden_slides': hidden_slides,
                        'presentation_metadata': presentation_metadata  # Store metadata in checkpoint
                    }
                    with open(checkpoint_file, 'w') as f:
                        json.dump(checkpoint_data, f)
                    print(f"Saved checkpoint after slide {slide_num}")

                    # Replace previous temp file if it exists
                    prev_temp = f'temp_{Path(pptx_file).stem}_slide_{slide_num-1}.pptx'
                    if Path(prev_temp).exists():
                        Path(prev_temp).unlink()

            except Exception as e:
                print(f"Error processing slide {slide_num}: {str(e)}")
                continue

        # Rehide hidden slides
        for slide_num in presentation_metadata.get('hidden_slides', []):
            presentation.slides[slide_num-1]._element.set("show", "0")

        # Save final version
        output_pptx = f'{Path(pptx_file).stem}_{start_slide}_{end_slide}.pptx'
        presentation.save(output_pptx)
        print(f"Saved final presentation: {output_pptx}")

        # Clean up checkpoint and temp files
        Path(checkpoint_file).unlink(missing_ok=True)
        for temp_file in Path().glob(f'temp_{Path(pptx_file).stem}_slide_*.pptx'):
            temp_file.unlink()

        print(f"Processed {total_images} images across slides {start_slide}-{end_slide}")
        print(f"Added {titles_added} hidden titles to slides missing titles")
        print(f"Classifications: {classifications}")
        return output_pptx

    except Exception as e:
        print(f"Error processing presentation: {str(e)}")
        return None


# Word Documents
Expand this section to input a Word Document.

## With User Input
Run the code cell below to upload a `.docx` file and to be able to manually change the generated alt text.

In [ ]:
from google.colab import files
uploaded = files.upload()

for docx_file in uploaded.keys():
    print(f"Processing {docx_file}...")
    output_file = process_docx(docx_file)
    print(f"Completed processing. Updated file saved as {output_file}")
    files.download(output_file)

## Fully Automated
Run the code cell below to upload a `.docx` file and let the process run uninterrupted.

In [ ]:
from google.colab import files
uploaded = files.upload()

for docx_file in uploaded.keys():
    print(f"Processing {docx_file}...")
    output_file = process_docx(docx_file, user_input=False)
    print(f"Completed processing. Updated file saved as {output_file}")
    files.download(output_file)

# PowerPoint Documents
Expand this section to input a PowerPoint document.

## With User Input
Expand this section to upload a `.pptx` file and to be able to manually change the generated alt text.

### Full File
Run the code cell below to upload a `.pptx` file and process it fully.

In [ ]:
from google.colab import files
uploaded = files.upload()

for pptx_file in uploaded.keys():
    print(f"Processing {pptx_file}...")
    classifications = {}
    output_file = process_pptx(pptx_file, start_slide=1, end_slide=50, classifications=classifications)
    presentation = Presentation(pptx_file)
    num_slides = len(presentation.slides)
    file = pptx_file
    start = 51
    end = 100
    while start <= num_slides:
      file = f"{file[:file.rindex('.')]}_{start - 50}_{end - 50}{file[file.rindex('.'):]}"
      print(f"Processing {file}...")
      output_file = process_pptx(file, start_slide=start, end_slide=end, classifications=classifications)
      start += 50
      end += 50
    file = f"{file[:file.rindex('.')]}_{start - 50}_{num_slides}{file[file.rindex('.'):]}"
    os.rename(file, f"updated_{pptx_file}")
    file = f"updated_{pptx_file}"
    print(f"Classifications: {classifications}")
    files.download(file)

Run the code cell below to perform a second pass of alt text generation.

In [ ]:
for pptx_file in uploaded.keys():
    classifications = {}
    file = pptx_file
    #file = f"{file[:file.rindex('.')]}_{start - 50}_{num_slides}{file[file.rindex('.'):]}"
    file = f"updated_{pptx_file}"
    print(f"Reprocessing {file}...")
    output_file = process_pptx(file, start_slide=1, end_slide=50, reprocess=True, classifications=classifications)
    start = 51
    end = 100
    while start <= num_slides:
      file = f"{file[:file.rindex('.')]}_{start - 50}_{end - 50}{file[file.rindex('.'):]}"
      print(f"Reprocessing {file}...")
      output_file = process_pptx(file, start_slide=start, end_slide=end, reprocess=True, classifications=classifications)
      start += 50
      end += 50
    file = f"{file[:file.rindex('.')]}_{start - 50}_{num_slides}{file[file.rindex('.'):]}"
    os.rename(file, f"updated_{pptx_file}")
    file = f"updated_{pptx_file}"
    print(f"Classifications: {classifications}")
    files.download(file)
    pptx_to_images_result = None

### Specific Range
Run the two code cells below to upload a `.pptx` file and process only a specific range of slides.

Enter the start and end slides you wish to process.

In [ ]:
#enter start slide here
start_slide = 10

#enter end slide here
end_slide = 15

In [ ]:
from google.colab import files
uploaded = files.upload()

for pptx_file in uploaded.keys():
    print(f"Processing {pptx_file}...")
    classifications = {}
    output_file = process_pptx(pptx_file, start_slide=start_slide, end_slide=end_slide, classifications=classifications)
    print(f"Classifications: {classifications}")
    classifications = {}
    file = process_pptx(output_file, start_slide=start_slide, end_slide=end_slide, reprocess=True, classifications=classifications)
    print(f"Classifications: {classifications}")
    os.rename(file, f"updated_{pptx_file}")
    file = f"updated_{pptx_file}"
    files.download(file)
    pptx_to_images_result = None

## Fully Automated
Expand this section to upload a `.pptx` file and let the process run uninterrupted.

### Full File
Run the code cell below to upload a `.pptx` file and process it fully.

In [ ]:
from google.colab import files
uploaded = files.upload()

for pptx_file in uploaded.keys():
    print(f"Processing {pptx_file}...")
    classifications = {}
    output_file = process_pptx(pptx_file, start_slide=1, end_slide=50, user_input=False, classifications=classifications)
    presentation = Presentation(pptx_file)
    num_slides = len(presentation.slides)
    file = pptx_file
    start = 51
    end = 100
    while start <= num_slides:
      file = f"{file[:file.rindex('.')]}_{start - 50}_{end - 50}{file[file.rindex('.'):]}"
      print(f"Processing {file}...")
      output_file = process_pptx(file, start_slide=start, end_slide=end, user_input=False, classifications=classifications)
      start += 50
      end += 50
    file = f"{file[:file.rindex('.')]}_{start - 50}_{num_slides}{file[file.rindex('.'):]}"
    os.rename(file, f"updated_{pptx_file}")
    file = f"updated_{pptx_file}"
    print(f"Classifications: {classifications}")
    files.download(file)

Run the code cell below to perform a second pass of alt text generation.

In [ ]:
for pptx_file in uploaded.keys():
    classifications = {}
    file = pptx_file
    #file = f"{file[:file.rindex('.')]}_{start - 50}_{num_slides}{file[file.rindex('.'):]}"
    file = f"updated_{pptx_file}"
    print(f"Reprocessing {file}...")
    output_file = process_pptx(file, start_slide=1, end_slide=50, user_input=False, reprocess=True, classifications=classifications)
    start = 51
    end = 100
    while start <= num_slides:
      file = f"{file[:file.rindex('.')]}_{start - 50}_{end - 50}{file[file.rindex('.'):]}"
      print(f"Reprocessing {file}...")
      output_file = process_pptx(file, start_slide=start, end_slide=end, reprocess=True, user_input=False, classifications=classifications)
      start += 50
      end += 50
    file = f"{file[:file.rindex('.')]}_{start - 50}_{num_slides}{file[file.rindex('.'):]}"
    os.rename(file, f"updated_{pptx_file}")
    file = f"updated_{pptx_file}"
    print(f"Classifications: {classifications}")
    files.download(file)
    pptx_to_images_result = None

### Specific Range
Run the two code cells below to upload a `.pptx` file and process only a specific range of slides.

Enter the start and end slides you wish to process.

In [ ]:
#enter start slide here
start_slide = 3

#enter end slide here
end_slide = 3

In [ ]:
from google.colab import files
uploaded = files.upload()

for pptx_file in uploaded.keys():
    print(f"Processing {pptx_file}...")
    classifications = {}
    output_file = process_pptx(pptx_file, start_slide=start_slide, end_slide=end_slide, user_input=False, classifications=classifications)
    print(f"Classifications: {classifications}")
    classifications = {}
    file = process_pptx(output_file, start_slide=start_slide, end_slide=end_slide, reprocess=True, user_input=False, classifications=classifications)
    print(f"Classifications: {classifications}")
    os.rename(file, f"updated_{pptx_file}")
    file = f"updated_{pptx_file}"
    files.download(file)
    pptx_to_images_result = None